# Import & Load Childhood data (Under-One Year)

### Config

In [1]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open("../../../settings.ini"))

sys.path.insert(0, config.get("PATHS", "libs_path"))
engine_path = config.get("DATABASE", "engine_path")

### Main params

In [4]:
remote_path = ""
local_path = "../data_final/"

In [5]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [6]:
#Open unzipped file
df = pd.read_csv(local_path + "ive_junaeb.csv", delimiter=",")
# df = df[['year','comuna_id','sex_id','rate_comuna','rate_region','rate_country','count','age_range_id']]
# df = df.astype({"year":"int", "comuna_id": "int",'sex_id':'int','rate_comuna':'float','rate_region':'float','rate_country':'float', 'age_range_id': 'float'})
list(df)

['Unnamed: 0',
 'rbd',
 'administration',
 'zone_id',
 'comuna_datachile_id',
 'education',
 'year',
 'priority',
 'total']

### Ingest

In [7]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, "childhood", "fact_vulnerability")

/Users/cnavarreteliz/Datawheel/datachile-etl/libs/postgres.py:30: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql_str = get_schema(dataframe, table_name, dtype=dtype)


DROP TABLE IF EXISTS childhood.fact_vulnerability;
CREATE TABLE "childhood"."fact_vulnerability" (
"Unnamed: 0" INTEGER,
  "rbd" INTEGER,
  "administration" INTEGER,
  "zone_id" INTEGER,
  "comuna_datachile_id" INTEGER,
  "education" INTEGER,
  "year" INTEGER,
  "priority" INTEGER,
  "total" INTEGER
)
COPY "childhood"."fact_vulnerability" ("Unnamed: 0","rbd","administration","zone_id","comuna_datachile_id","education","year","priority","total") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Add Indices to foreign keys

In [9]:
engine.execute("""
CREATE INDEX fact_vulnerability_comuna_id_index 
ON childhood.fact_vulnerability (comuna_id)
""")

engine.execute("""
CREATE INDEX fact_vulnerability_year_index 
ON childhood.fact_vulnerability ("year");
""")

ProgrammingError: (psycopg2.ProgrammingError) column "comuna_id" does not exist
 [SQL: '\nCREATE INDEX fact_vulnerability_comuna_id_index \nON childhood.fact_vulnerability (comuna_id)\n'] (Background on this error at: http://sqlalche.me/e/f405)